In [ ]:
# 파일 경로 설정
file_path = '/Users/dryoon04/Documents/GitHub/schoolproject/vectorstore/open ai api key.txt'

# 파일 열기 (읽기 모드로 열기)
with open(file_path, 'r', encoding='utf-8') as file:
    # 파일 내용 읽어오기
    file_content = file.read()

In [ ]:
import pandas as pd
import chromadb
import os
import openai


In [ ]:
df = pd.read_csv('lyrics.csv')

In [ ]:
client = chromadb.PersistentClient("chromadb/lyrics.db")
collection = client.get_or_create_collection(name="lyrics",metadata={"hnsw:space":"cosine"})


In [ ]:
singers = []
song_titles = []
lyrics = []
ids = []
documents = []

for idx in range(len(df)):
    item = df.iloc[idx]
    singer = item['가수'].strip().lower()  # 가수 정보
    song_title = item['제목'].strip().lower()  # 노래 제목 정보
    lyric = item['가사'].strip().lower()  # 가사 정보

    # id = f"{singer}-{song_title}".replace(' ', '-')  # 고유한 ID 생성
    id = f"{singer}-{song_title}"
    document = f"{singer}:{song_title}:{lyric}"  # 문서 텍스트 생성
    
    
    ids.append(id)
    documents.append(document)

# DB 저장
collection.add(documents=documents,ids=ids)


In [ ]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(user_query)
print(vector_res)

In [ ]:

os.environ["OPENAI_API_KEY"] = file_content
openai.api_key = os.getenv("OPENAI_API_KEY")
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(vector_res)
srchres = []
for v in vector_res['documents'][0]:
    item = v.split(':')
    srchres.append({"가수" : item[0].strip(),"제목" : item[1].strip(),"가사" : item[2].strip()})
    
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
               {"role": "system","content": "You are an assistant who can answer questions about music lyricsBased on the following database In order to answer questions, you should only use facts from the databaseIf you feel you don't have enough information to answer a question, you should say, '답할수있는 정보가 없습니다.'."},
            {"role": "user", "content": f"{user_query}"},
            {"role": "assistant", "content": f"{srchres}"},
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    
print(completion["choices"][0]["message"]["content"])


In [ ]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=10)
print(user_query)
print(vector_res)